<a href="https://colab.research.google.com/github/robsonbfreitas/CalculadoraDeAluguel/blob/main/TratamentoDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
url = 'https://www.vivareal.com.br/venda/sp/sao-paulo/apartamento_residencial/'

In [4]:
c = requests.get(url).content

In [5]:
soup = BeautifulSoup(c)

In [6]:
aluguel_tag = soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'})

In [7]:
len(aluguel_tag)
paginas = ['']
for i in range(2,40):
    a = '?pagina='+str(i)
    print(a)
    paginas.append(a)


?pagina=2
?pagina=3
?pagina=4
?pagina=5
?pagina=6
?pagina=7
?pagina=8
?pagina=9
?pagina=10
?pagina=11
?pagina=12
?pagina=13
?pagina=14
?pagina=15
?pagina=16
?pagina=17
?pagina=18
?pagina=19
?pagina=20
?pagina=21
?pagina=22
?pagina=23
?pagina=24
?pagina=25
?pagina=26
?pagina=27
?pagina=28
?pagina=29
?pagina=30
?pagina=31
?pagina=32
?pagina=33
?pagina=34
?pagina=35
?pagina=36
?pagina=37
?pagina=38
?pagina=39


In [8]:
import pandas as pd

In [9]:
def converter_df(aluguel_lista,quartos_lista,endereco_lista,banheiro_lista,vaga_lista,area_lista):
    data = {'aluguel':aluguel_lista,
           'quartos':quartos_lista,
           'endereco':endereco_lista,
           'banheiro':banheiro_lista,
            'vaga':vaga_lista,
            'area':area_lista}
    df = pd.DataFrame(data)
    return df

In [10]:
def scrap(url):
    c = requests.get(url).content
    soup = BeautifulSoup(c)
    
    aluguel_tag = soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'}) 
    aluguel_lista = [p.text for p in aluguel_tag] 
    #print(len(aluguel_lista)) 
    
    quartos_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-room js-property-detail-rooms'}) 
    quartos_lista = [en.text for en in quartos_tag] 
    #print(len(quartos_lista)) 
    
    endereco_tag = soup.find_all('span', {'class': 'property-card__address'}) 
    endereco_lista = [fg.text for fg in endereco_tag] 
    #print(len(endereco_lista)) 
    
    banheiro_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'}) 
    banheiro_lista = [yp.text for yp in banheiro_tag] 
    #print(len(banheiro_lista))

    vaga_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-garage js-property-detail-garages'}) 
    vaga_lista = [yp.text for yp in vaga_tag] 

    area_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-area'}) 
    area_lista = [yp.text for yp in area_tag] 


    ### Vamos ter criar uma função para transformar listas em df
    df=converter_df(aluguel_lista,quartos_lista,endereco_lista,banheiro_lista,vaga_lista,area_lista)
    
    return df


In [11]:
URL = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/{}'

In [12]:
df_lista = []
for pg in paginas:
    url = URL.format(pg)
    print('Coletando URL:', url)
    try:
        df = scrap(url)
    except Exception as e:
        print('Erro ao coletar dados na url:', url)
        print(e)
    df_lista.append(df)

Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=2
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=3
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=4
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=5
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=6
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=7
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=8
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=9
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=10
Coletando URL: https://www.v

In [42]:
df_final = pd.concat(df_lista)

In [43]:
df_final.head()

,aluguel,quartos,endereco,banheiro,vaga,area
0,R$ 2.500 /Mês,1 Quarto,"Rua Castro Alves, 265 - Aclimação, São Paulo - SP",1 Banheiro,1 Vaga,50 m²
1,R$ 3.000 /Mês,2 Quartos,"Rua Guararapes, 300 - Brooklin, São Paulo - SP",2 Banheiros,1 Vaga,76 m²
2,R$ 6.000 /Mês,2 Quartos,"Alameda Joaquim Eugênio de Lima, 711 - Jardim ...",3 Banheiros,2 Vagas,80 m²
3,R$ 2.000 /Mês,1 Quarto,"Avenida Divino Salvador - Planalto Paulista, S...",1 Banheiro,1 Vaga,40 m²
4,R$ 2.050 /Mês,3 Quartos,"Avenida Marechal Mário Guedes, 02 - Jaguaré, S...",2 Banheiros,2 Vagas,70 m²


In [44]:
df_final['bairro'] = df_final.endereco.str.split('\s-').str[1]
df_final['rua'] = df_final.endereco.str.split('\s-').str[0]
df_final['bairro'] = df_final.bairro.str.split(',').str[0]

In [45]:
df_final.head()

,aluguel,quartos,endereco,banheiro,vaga,area,bairro,rua
0,R$ 2.500 /Mês,1 Quarto,"Rua Castro Alves, 265 - Aclimação, São Paulo - SP",1 Banheiro,1 Vaga,50 m²,Aclimação,"Rua Castro Alves, 265"
1,R$ 3.000 /Mês,2 Quartos,"Rua Guararapes, 300 - Brooklin, São Paulo - SP",2 Banheiros,1 Vaga,76 m²,Brooklin,"Rua Guararapes, 300"
2,R$ 6.000 /Mês,2 Quartos,"Alameda Joaquim Eugênio de Lima, 711 - Jardim ...",3 Banheiros,2 Vagas,80 m²,Jardim Paulista,"Alameda Joaquim Eugênio de Lima, 711"
3,R$ 2.000 /Mês,1 Quarto,"Avenida Divino Salvador - Planalto Paulista, S...",1 Banheiro,1 Vaga,40 m²,Planalto Paulista,Avenida Divino Salvador
4,R$ 2.050 /Mês,3 Quartos,"Avenida Marechal Mário Guedes, 02 - Jaguaré, S...",2 Banheiros,2 Vagas,70 m²,Jaguaré,"Avenida Marechal Mário Guedes, 02"


In [46]:
df_final['quartos'] = df_final.quartos.str.extract('(.*) Quarto')
df_final['banheiro'] = df_final.banheiro.str.extract('(.*) Banheiro')
df_final['vaga'] = df_final.vaga.str.extract('(.*) Vaga')
df_final['area'] = df_final.area.str.extract('(.*)  m²')

In [47]:
df_final.head()

,aluguel,quartos,endereco,banheiro,vaga,area,bairro,rua
0,R$ 2.500 /Mês,1,"Rua Castro Alves, 265 - Aclimação, São Paulo - SP",1,1,50,Aclimação,"Rua Castro Alves, 265"
1,R$ 3.000 /Mês,2,"Rua Guararapes, 300 - Brooklin, São Paulo - SP",2,1,76,Brooklin,"Rua Guararapes, 300"
2,R$ 6.000 /Mês,2,"Alameda Joaquim Eugênio de Lima, 711 - Jardim ...",3,2,80,Jardim Paulista,"Alameda Joaquim Eugênio de Lima, 711"
3,R$ 2.000 /Mês,1,"Avenida Divino Salvador - Planalto Paulista, S...",1,1,40,Planalto Paulista,Avenida Divino Salvador
4,R$ 2.050 /Mês,3,"Avenida Marechal Mário Guedes, 02 - Jaguaré, S...",2,2,70,Jaguaré,"Avenida Marechal Mário Guedes, 02"
